In [1]:
import sys
sys.path.append('../')

In [1]:
from AoE2ScenarioParser.datasets.players import PlayerId
from AoE2ScenarioParser.datasets.units import UnitInfo
from AoE2ScenarioParser.datasets.buildings import BuildingInfo
from AoE2ScenarioParser.datasets.other import OtherInfo
from AoE2ScenarioParser.datasets.terrains import TerrainId
from aoe2mapgenerator.common.enums.enum import (
    MapLayerType, 
    ObjectSize, 
    GateTypes, 
    TemplateTypes, 
    ObjectRotation, 
    YamlReplacementKeywords,
    CheckPlacementReturnTypes
)

from aoe2mapgenerator.scenario.scenario import Scenario
import numpy as np
import random
from time import time
from aoe2mapgenerator.common.constants.constants import DEFAULT_EMPTY_VALUE, BASE_SCENE_DIR
from aoe2mapgenerator.common.enums.enum import GateTypes
import multiprocessing as mp
from aoe2mapgenerator.map.map import Map
import os

In [2]:
input_name = "1 vs 7 default.aoe2scenario"
output_name = "0_BASIC_SCENARIO.aoe2scenario"

In [3]:
# start_time = time()
# # result = np.array(lst)
# lst = np.array(lst)
# end_time = time()
# print(end_time-start_time)

In [4]:
map_size = 256
map = Map(map_size)
new_zones = map.voronoi(75,
            [MapLayerType.UNIT, MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.DECOR],
            [DEFAULT_EMPTY_VALUE, DEFAULT_EMPTY_VALUE, DEFAULT_EMPTY_VALUE, DEFAULT_EMPTY_VALUE])

keys = list(map.get_map_layer(MapLayerType.UNIT).dict.keys())

for city_zone in keys:
    map.add_borders(
        [MapLayerType.TERRAIN, MapLayerType.UNIT, MapLayerType.ZONE, MapLayerType.DECOR],
        [city_zone, city_zone, city_zone,city_zone],
        TerrainId.ROAD_FUNGUS,
        margin = 2,
        )

In [5]:
def build_city(zone, player_id):
    """
    Build city in zone
    """
    print("BUILD CITY")

    map.add_borders(
        [MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.UNIT, MapLayerType.DECOR],
        [zone, zone, zone, zone],
        TerrainId.GRASS_2,
        margin = 10
        )

    map.place_template(
            'oak_forest.yaml',
            map_layer_type_list = [MapLayerType.UNIT, MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.DECOR],
            array_space_type_list = [zone, (TerrainId.GRASS_2, PlayerId.GAIA), zone, zone],
            player_id = player_id,
        )

    map.place_template(
            'walls.yaml',
            map_layer_type_list = [MapLayerType.UNIT, MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.DECOR],
            array_space_type_list = [zone, zone, zone, zone],
            player_id = player_id,
        )
    
    map.add_borders(
        [MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.UNIT, MapLayerType.DECOR],
        [zone, zone, zone, zone],
        TerrainId.ROAD_FUNGUS,
        margin = 1
        )
    
    city_zones = map.voronoi(35,
                [MapLayerType.UNIT, MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.DECOR],
                [zone, zone, zone, zone],
            )
    
    for city_zone in city_zones:
        map.add_borders(
            [MapLayerType.TERRAIN, MapLayerType.UNIT, MapLayerType.ZONE, MapLayerType.DECOR],
            [city_zone, city_zone, city_zone, city_zone],
            TerrainId.ROAD_FUNGUS,
            margin = 1
            )
        
        map.place_template(
            'oak_forest.yaml',
            map_layer_type_list = [MapLayerType.UNIT, MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.DECOR],
            array_space_type_list = [city_zone, city_zone, city_zone, city_zone],
        )

        map.place_template(
            'City.yaml',
            map_layer_type_list = [MapLayerType.UNIT, MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.DECOR],
            array_space_type_list = [city_zone, city_zone, city_zone, city_zone],
            player_id = player_id,
        )

In [6]:
print(BASE_SCENE_DIR)

C:\Users\josep\Games\Age of Empires 2 DE\76561198242754748\resources\_common\scenario\


In [7]:
def build_snow_forest(zone, player_id):
    """
    Build snow forest in zone
    """
    print("BUILD FOREST")
    
    map.place_template(
            'snow_forest.yaml',
            map_layer_type_list = [MapLayerType.UNIT, MapLayerType.TERRAIN, MapLayerType.ZONE, MapLayerType.DECOR],
            array_space_type_list = [zone, zone, zone, zone],
            player_id = player_id,
        )

In [8]:
counter = 0
for i, zone in enumerate(new_zones):
    # print(zone)
    counter += 1
    if counter >= 9:
        counter = 1
    
    if random.random() > 0.5:
        build_city(zone, PlayerId(counter))
    else:
        build_snow_forest(zone, PlayerId(counter))
        

BUILD FOREST
NEW TEMPLATE LOADED: snow_forest.yaml
The value (<TerrainId.GRASS_2: 12>, <PlayerId.GAIA: 0>) is not present in the MapLayerType.TERRAIN map.
BUILD CITY
NEW TEMPLATE LOADED: oak_forest.yaml
NEW TEMPLATE LOADED: walls.yaml
NEW TEMPLATE LOADED: City.yaml
BUILD FOREST
BUILD CITY
BUILD FOREST
BUILD FOREST
BUILD CITY
BUILD FOREST
BUILD FOREST
BUILD CITY
BUILD FOREST


In [9]:
map.visualize_mat(MapLayerType.UNIT, include_zones=False)

In [ ]:
scenario = Scenario(map)


[22:59:50] Reading file: 'C:\Users\josep\Games\Age of Empires 2 DE\76561198242754748\resources\_common\scenario\BASE_SCENARIO.aoe2scenario'
[22:59:50] Reading scenario file finished successfully.

############### Attributes ###############
>>> Game version: 'DE'
>>> Scenario version: 1.51
>>> Scenario variant: 'Age of Empires 2'
##########################################

[22:59:50] Loading scenario structure...

UnknownScenarioStructureError: The version DE:1.51 is not supported by AoE2ScenarioParser. :(

In [ ]:
scenario.change_map_size(map_size)
scenario.write_map()
scenario.save_file(os.path.join(BASE_SCENE_DIR, "TestScenario2.aoe2scenario"))


[22:57:43] Reconstructing sections and structs from managers...
	✔ MessageManagerssageManager...
	✔ PlayerManagerlayerManager...
	✔ MapManagerg MapManager...
	✔ UnitManager UnitManager...
	✔ TriggerManageriggerManager...
	✔ XsManagerng XsManager...
[22:57:45] Reconstruction finished successfully.

[22:57:45] File writing from structure started...
	✔ FileHeaderg FileHeader...
	✔ DataHeaderg DataHeader...
	✔ Messagesing Messages...
	✔ Cinematicsg Cinematics...
	✔ BackgroundImagekgroundImage...
	✔ PlayerDataTwolayerDataTwo...
	✔ GlobalVictorylobalVictory...
	✔ Diplomacyng Diplomacy...
	✔ Optionsting Options...
	✔ Maptructing Map...
	✔ Unitsucting Units...
	✔ Triggersing Triggers...
	✔ Filesucting Files...
[22:57:46] File writing finished successfully.
[22:57:46] File successfully written to: 'C:\Users\josep\Games\Age of Empires 2 DE\76561198242754748\resources\_common\scenario\TestScenario2.aoe2scenario'
[22:57:46] Execution time from scenario read: 8.08s


In [ ]:
import tkinter as tk

n = 10
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'white']

def on_color_click(color):
    global selected_color
    selected_color = color

def on_square_click(row, col):
    btn = buttons[row][col]
    # print(btn)
    # print(type(btn))
    if btn.cget('bg') == selected_color:
        btn.config(bg='white')
    else:
        btn.config(bg=selected_color)

default_color = 'white'

root = tk.Tk()

frame = tk.Frame(root)
frame.pack(side=tk.LEFT)

buttons = []
for i in range(n):
    row = []
    for j in range(n):
        btn = tk.Button(frame, text='    ', command=lambda i=i, j=j: on_square_click(i, j), bg = default_color)
        btn.grid(row=i, column=j)
        row.append(btn)
    buttons.append(row)

color_frame = tk.Frame(root)
color_frame.pack(side=tk.RIGHT)

for color in colors:
    color_label = tk.Label(color_frame, text=color)
    color_label.pack(side=tk.BOTTOM)
    btn = tk.Button(color_frame, bg=color, command=lambda color=color: on_color_click(color))
    btn.pack(side=tk.BOTTOM)

selected_color = colors[0]

root.mainloop()

In [4]:
import tkinter as tk

root = tk.Tk()

# create three labels
label1 = tk.Label(root, text="Label 1")
label2 = tk.Label(root, text="Label 2")
label3 = tk.Label(root, text="Label 3")

# pack the first two labels to the left
label1.pack(side="left")
label2.pack(side="left")

# pack the third label to the right
label3.pack(side="right")

root.mainloop()